# 3. SciPyのスパースマトリクス入門

## 目次

- [3.1 はじめに：スパースマトリクスについて](#what_is_sparse_matrix)
    - 3.1.1. SciPyのsparseクラスのインポート
- [3.2 coo_matrix](#coo)
- [3.3 csr_matrix / csc_matrix](#csr_csc)
- [3.4 各スパースマトリクスの長所と短所](#ad_disad)
- [3.5 sparse matrixの基本操作](#basic_operation)
- [3.6 まとめ](#summary)

<a id='what_is_sparse_matrix'></a>

## 3.1 はじめに：スパースマトリクスについて

ここまで，`np.ndarray`の扱いに関して学んできました．`np.ndarray`を用いることで，行列の様々な演算を簡単に・高速に行うことが可能になりました．

さて，実際のデータ解析において**行列の殆どの要素が0である**ということがあります（本演習で用いるデータもまさしくそうです）．要素の殆どが0である状態のことを**スパース (Sparse, 疎)**であると言います．スパースな行列に対しても`np.ndarray`を用いることができますが，**スパースな行列のためのデータ構造**を用いることで**より少ないメモリでデータを表現**でき，さらに**より高速に演算**を行うことが可能になります．このことを確かめるために，いくつかの演算について行列がスパースな場合について考えてみましょう．

1. **行列(ベクトル)の要素の和** `np.sum(X)` : 和というのは足し算で，ある値に0を足しても値は変わりません．したがって，和を求める場合は**`X`の0でない値だけを記憶しておけば十分**です．仮に`X`がスパースで99%の要素が0だとすると，高々1%の値のみ保持しておけばよく，大幅にメモリを削減でき更に和の計算も大幅に高速化されます．
2. **ベクトルの内積** `np.dot(a, b)` :  ベクトル`b`がスパースだとします．内積は要素ごとの積の和，すなわち，\$\sum_{i=1}^{n}\$`a[i]*b[i]`です．`b`がスパースということは，`b[i]`のほとんどが0ということで，すなわち`a[i]*b[i]`のほとんどが0ということです．内積は`a[i]*b[i]`の和ですから，先ほどの行列の和の時と同様に，`a[i]*b[i]=0`となるような箇所は全て無視できます．したがって，**`b`の0でない値と，そのインデックスのみ保持していれば内積は計算できます**．

これらのことから，行列全体ではなく**行列の0でない要素の値とそのインデックス**のみ保持しておけば，メモリ・時間の効率を向上させながら様々な演算を通常の行列と同じように実行可能だということがわかります．このようなデータ構造のことを**スパースマトリクス (sparse matrix）**と言い，これはSciPyで提供されています．スパースマトリクスにもいくつか種類があります．それらの違いはインデックスの保持の仕方によって生じており，それぞれ得意な演算が異なります．

本Notebookでは，`coo_matrix`・`csr_matrix`・`csc_matrix`の主に3つについて紹介します．また，いくつかの関数についても紹介します．詳しくは公式のリファレンス https://docs.scipy.org/doc/scipy/reference/sparse.html をご覧ください．

**注意** スパースマトリクスはその名の通り行列を扱うクラスです．残念ながら一般の配列には対応していません．

## 3.1.1 SciPyのsparseクラスのインポート

SciPyのパッケージ名は`scipy`です．スパースマトリクスはその中の`sparse`クラスに属しています．以下の一文でスパースマトリクスを使うことが可能になります．

In [1]:
from scipy import sparse
import numpy as np # numpyも用いるのでインポートしておく

これ以降，`sparse.hugahuga`という形でSciPyのスパースマトリクスが使えるようになりました (`import scipy`とし，`scipy.sparse.fugafuga`としても使えますが，冗長なのでこのようにしています)．実際に0の多い行列を作ってみた後，(1)`np.ndarray`を用いた場合，(2)`scipy.sparse.csc_matrix` (スパースマトリクスの一つです）を用いた場合の和演算の速度で比較をしてみます．

In [2]:
A = np.random.rand(1000*1000).reshape(1000, 1000) # (1000, 1000)の行列，各要素は0から1
A[A<0.99] = 0 # 0.99以下の値を全て0にする．期待値的には1%が非ゼロ，99%が0．
A_sp = sparse.csc_matrix(A) # aのスパースマトリクス表現

%timeit np.sum(A)
%timeit A_sp.sum()

834 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
147 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


流石に100倍とはなっていませんが，高速になったことがわかります．

メモリ使用量についてはどうでしょうか？比較してみましょう．`nbytes`でオブジェクトのバイト数が得られます．`np.ndarray`の方は，64bit×1000×1000/8=8000000バイトだと予想されます．`csc_matrix`では，値を表す`data`以外に，インデックスを表す`indices`と`indptr`の２つの属性があります（これらが何を表すかは後ほど説明します）．これら全てのバイト数を計算し，その合計を`csc_matrix`のメモリ使用量とします．

In [3]:
memory_ndarray = A.data.nbytes
memory_cscmat = A_sp.data.nbytes + A_sp.indices.nbytes + A_sp.indptr.nbytes
print(memory_ndarray, memory_cscmat)
print(memory_ndarray/memory_cscmat)

8000000 125336
63.82842918235782


`ndarray`の場合と比べ約60分の1となっています，素晴らしいですね．では実際にいくつかの疎行列について見ていきましょう．

<a id='coo'></a>

## 3.2 coo_matrix

スパースな行列として次のような行列を例に考えることにします（実際に扱うデータはもっとスパース（非ゼロ要素が1%未満）であることも多いです）．

In [4]:
A_list = [[0, 0, 1, 0], [0, 4, -1, 0], [2, 0, -3, 0], [0, 0, 0, 5]]
A_ndarray = np.array(A_list)
print(A_ndarray)

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]


先程述べたように，スパースな行列は全ての値ではなく非ゼロ要素の値とそのインデックスを覚えておけば良いのでした．上で定義した行列`A`の非ゼロ要素の値とインデックス（何行・何列か）は，

値　インデックス

` 1 (0, 2)`
 
` 4 (1, 1)`
 
` -1 (1, 2)`
 
` 2 (2, 0)`

` -3 (2, 2)`

` 5 (3, 3)`

です．スパースマトリクスとしての最もシンプルな表現方法は，行列`A`の(1)非ゼロ要素の値を表すリスト (あるいは`np.ndarray`)，(2)それらの行のインデックスを表すリスト，そして(3)列のインデックスを表すリストの合計3つのリストによって表現することです．すなわち，

`data = [1, 4, -1, 2, -3, 5] # 非ゼロ要素の値のリスト`

`row = [0, 1, 1, 2, 2, 3] # 非ゼロ要素の行のインデックスのリスト`

`col = [2, 1, 2, 0, 2, 3] # 非ゼロ要素の列のインデックスのリスト`

の3つのリストの集まりとして行列を表現するということです．このように表現した場合，**非ゼロ要素の3倍のサイズのデータ**を保持することになります．したがって，単純に考えると**非ゼロ要素が全体の1/3より少ない時は効率が良い**ですが，そうでない場合は効率は良くありません．この例では残念ながら効率は良くありません．

このような表現で計算は効率良く行えるのでしょうか？まず，和の計算は`data`の和を計算すれば良く，これは非ゼロ要素に比例した計算時間なため，`np.ndarray`上で行うよりも効率が良いでしょう．また，同じサイズの行列`B`との内積 (要素ごとの積の和，すなわち，NumPyの場合は`np.sum(A*B)`)を考えてみます．これもやはり，\$\sum_{i \in \mathrm{row}, j \in \mathrm{col}}\$ `A[i,j]*B[i,j]`で計算できる (2重ループを回す際に，`row`と`col`内のインデックスだけ取り出せば良い）ため，非ゼロ要素に比例した計算時間で実行できます．

非ゼロ要素の値のリスト・非ゼロ要素の行のインデックスのリスト・非ゼロ要素の列のインデックスのリストの3つで表現するスパースマトリクスを，`scipy.sparse`では`coo_matrix`として提供しています (COOrdinate (座標) format)．実際に使ってみましょう．最も単純な作り方は，`np.ndarray`を`coo_matrix`に被せるという方法です．

In [5]:
A_coo = sparse.coo_matrix(A_ndarray)
print(A_coo)
print(A_coo.data)
print(A_coo.row)
print(A_coo.col)

  (0, 2)	1
  (1, 1)	4
  (1, 2)	-1
  (2, 0)	2
  (2, 2)	-3
  (3, 3)	5
[ 1  4 -1  2 -3  5]
[0 1 1 2 2 3]
[2 1 2 0 2 3]


この`np.ndarray`を被せるという方法は実用上は問題があります．行列が非常に大規模である場合に，`np.ndarray`ではそもそもメモリに乗らないということもあるからです．このような場合に対して，`data`・`row`・`col`の3つの配列を，`(data, (row, col))`の形で渡して`coo_matrix`を作ることもできます．3つのリストを`coo_matrix`に渡して`coo_matrix`のオブジェクトを作成することで，`scipy.sparse`の提供する様々な行列演算のサポートを受けることができます．

In [6]:
data = [1, 4, -1, 2, -3, 5]
row = [0, 1, 1, 2, 2, 3]
col = [2, 1, 2, 0, 2, 3]
A_coo2 = sparse.coo_matrix((data, (row, col)))
print(A_coo2)

  (0, 2)	1
  (1, 1)	4
  (1, 2)	-1
  (2, 0)	2
  (2, 2)	-3
  (3, 3)	5


ライブラリに実装されているため書く必要は全く無いのですが，coo_matrixの和を求める関数と，coo_matrixとndarrayの内積を求める関数は，例えば以下のように書けます．上手く動いています．後ライブラリの機能を用いた方法は後述します．

In [7]:
def sum_coo(X_coo):
    return np.sum(X_coo.data)


def dot_coo_ndarray(X_coo, Y_ndarray):
    # 愚直に書くと以下のようになる
    dot = 0.
    for val, i, j, in zip(X_coo.data, X_coo.row, X_coo.col): # 3つのリストから同時に取り出す
        dot += val * Y_ndarray[i, j]
    return dot
    """
    # インデックス配列によるアクセスを上手く使うと以下のように簡単に書ける
    return np.dot(X_coo.data, Y_ndarray[X_coo.row, X_coo.col])
    """

print(np.sum(A_ndarray)) # Aの和 (ndarray)
print(sum_coo(A_coo)) # Aの和 (coo_matrix)
print(np.sum(A_ndarray*A_ndarray)) # AとAの内積 (要素ごとの積の和) (ndarrayとndarray)
print(dot_coo_ndarray(A_coo, A_ndarray)) # AとAの内積 (要素ごとの積の和) (coo_matrixとndarray)

8
8
56
56.0


<a id='csr_csc'></a>

## 3.3 `csr_matrix / csc_matrix`

`coo_matrix`における行のインデックスリスト`row`と列のインデックスリスト`col`は，実はどちらか片方を簡略化することができます．先ほどの例では，上の行の値から順に値とインデックスをリストに入れていました．そのため，**同じ行に存在する値が連続して格納**されています．実際に，行のインデックスを表すリスト`row=[0, 1, 1, 2, 2, 3]`となっていることからも明らかです．

そこで，上の行から順に値が格納されている場合に，全ての非ゼロ要素の行のインデックスを陽に持つ`row`の代わりに，**`data`や`col`の何番目から何番目までが`i`行目かを表す**リストを持つことにしてみます．このようなリストを`indptr`と書くことにします（indexのpointer）．今，`indptr[i]`が，`data`や`col`において`i`行目の要素が格納され始めた場所を表すとします．`indptr[i+1]`は`i+1`行目の要素が格納され始めた場所ですから，`data[indptr[i]:indptr[i+1]]`が`i`行目の非ゼロ要素を表します．

わかりにくいので，実際に配列を表示しながら考えてみます．

In [8]:
print(A_ndarray)
print(A_coo.data)
print(A_coo.row)
print(A_coo.col)

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
[ 1  4 -1  2 -3  5]
[0 1 1 2 2 3]
[2 1 2 0 2 3]


`A`の0行目の要素は，`data`・`col`において0番目から開始します．したがって，`indptr[0]=0`です．

次に，`A`の1行目の要素は，0行目の非ゼロ要素は1つなので，`data`・`col`において1番目から開始します．したがって，`indptr[1]=1`です．

`A`の2行目の要素は，0行目と1行目の非ゼロ要素は3つなので，`data`・`col`において3番目から開始します．したがって，`indptr[2]=3`です．

最後に，`A`の3行目の要素は，0行目から2行目の非ゼロ要素は5つなので，`data`・`col`において5番目から開始します．したがって，`indptr[3]=5`です．

`indptr[i+1]`は，`i+1`行目の要素が始まる位置を表していますが，同時に`i`行目の要素がどこで終わるかを表してもいます．3行目の要素が何番目で終わるかを表すインデックスを格納している方が諸々の処理が綺麗に書けるため，`indptr[4]=6`とします．

まとめると，`indptr=[0, 1, 3, 5, 6]`となります．明らかに，`indptr`の要素数は行数+1で，これは基本的には非ゼロ要素数よりも少ないです（例えば，ゼロベクトルであるような行が多数ある場合は非ゼロ要素数よりも行数が多くなりえます）．したがって，非ゼロ要素数のサイズのリスト`row`でインデックスを保持するよりも効率が良くなります (この例では一つ減っただけですが)．

上の行から順に値を格納し，`data`・`col`においてある行の要素が何番目から始まっているかを表すリスト`indptr`を保持することでスパースな行列を表現するスパースマトリクスを，`scipy.sparse`では`csr_matrix`として提供しています (Compressed Sparse Row matrix)．`csr_matrix`では，`col`は`indices`という名前になります．

`csr_matrix`は行毎に行う演算・処理に適しています．
**`csr_matrix`では各行の非ゼロ要素には全体の非ゼロ要素数に依存しない時間でアクセスすることができます**．具体的には，`i`行目の非ゼロ要素は`data[indptr[i]:indptr[i+1]]`に格納されています．
一方，`coo_matrix`は，先ほどの例では上の行から順に格納しましたが，一般にはそうとは限りません．したがって，**`coo_matrix`ではある行の非ゼロ要素を取り出すためには全ての非ゼロ要素にアクセスする必要があります**（`i`行目の要素を全て取り出すためには，`row`の要素をすべてチェックして`i`行目の非ゼロ要素を探さなければならない）．

では実際に`csr_matrix`を作ってみましょう．これも，`np.ndarray`を被せることが作ることができます．
また，`coo_matrix`と同様に`(data, (row, col))`によっても作成できます．詳しくはリファレンスを参照してください．

In [9]:
A_csr = sparse.csr_matrix(A_ndarray)
print(A_csr)
print(A_csr.data)
print(A_csr.indptr)
print(A_csr.indices)

  (0, 2)	1
  (1, 1)	4
  (1, 2)	-1
  (2, 0)	2
  (2, 2)	-3
  (3, 3)	5
[ 1  4 -1  2 -3  5]
[0 1 3 5 6]
[2 1 2 0 2 3]


これもやはりライブラリに実装されているため書く必要は全く無いのですが，`csr_matrix`の和を求める関数と，`csr_matrix`と`ndarray`の内積を求める関数は，例えば以下のように書けます．上手く動いています．

In [10]:
def sum_csr(X_csr):
    return np.sum(X_csr.data)


def dot_csr_ndarray(X_csr, Y_ndarray):
    # 愚直に書くと以下のようになる
    dot = 0.
    indptr = X_csr.indptr
    indices = X_csr.indices
    data = X_csr.data
    
    for i in range(Y_ndarray.shape[0]):
        i_start = indptr[i]
        i_end = indptr[i+1]
        for val, j in zip(data[i_start:i_end], indices[i_start:i_end]):
            dot += val * Y_ndarray[i, j]
    return dot
    """
    # np.repeat・np.arange・インデックス配列によるアクセスを上手く使うと簡単に書ける
    reps = X_csr.indptr[1:] - X_csr.indptr[:-1]
    n_rows = Y_ndarray.shape[0]
    return np.dot(X_csr.data, Y_ndarray[np.repeat(np.arange(n_rows), reps), X_csr.indices])
    """

print(np.sum(A_ndarray)) # Aの和 (ndarray)
print(sum_csr(A_csr)) # Aの和 (coo_matrix)
print(np.sum(A_ndarray*A_ndarray)) # AとAの内積 (要素ごとの積の和) (ndarrayとndarray)
print(dot_csr_ndarray(A_csr, A_ndarray)) # AとAの内積 (要素ごとの積の和) (coo_matrixとndarray)

8
8
56
56.0


これまでは行の順に値を格納しました．一方で，列の順に値を格納するという方法も考えられます．すなわち，

`data = [2, 4, 1, -1, -3, 5] # 非ゼロ要素の値のリスト`

`row = [2, 1, 0, 1, 2, 3] # 非ゼロ要素の行のインデックスのリスト`

`col = [0, 1, 2, 2, 2, 3] # 非ゼロ要素の列のインデックスのリスト`

と，左の列から順に値を格納していく方法です．この場合，`col`がの要素が連続していますから，`csr_matrix`を作成した時の方法と同様の方法で，`col`を`indptr`に置き換えることができます．このようにして行列を表現するスパースマトリクスを，`scipy.sparse`では`csc_matrix` (Compressed Sparse Column matrix)として提供しています．`csr_matrix`では`col`が`indices`という名前になっていましたが，`csc_matrix`では`row`が`indices`という名前になっています．  


In [11]:
A_csc = sparse.csc_matrix(A_ndarray)
print(A_ndarray)
print(A_csc)
print(A_csc.data)
print(A_csc.indptr)
print(A_csc.indices)

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
  (2, 0)	2
  (1, 1)	4
  (0, 2)	1
  (1, 2)	-1
  (2, 2)	-3
  (3, 3)	5
[ 2  4  1 -1 -3  5]
[0 1 2 5 6]
[2 1 0 1 2 3]


`csr_matrix`は行毎に処理に適していましたが，`csc_matrix`は列毎に行う処理に適しています．理由は`csr_matrix`が行毎の処理に適しているのと同様の理由です：`csc_matrix`では各列の非ゼロ要素に「全体の非ゼロ要素数に依存しない時間」でアクセスが可能です．

<a id='ad_disad'></a>
## 3.4 各スパースマトリクスの長所・短所

ここまで，`coo_matrix`・`csc_matrix`・`csr_matrix`について紹介しました．これらの違いはインデックスの保持の仕方でした．
それぞれ長所と短所があります．
1. `coo_matrix`
    - 長所：他のスパースマトリクスに変換するのが高速．
    - 短所：計算が遅い，インデキシング・スライス等をサポートしていない．
2. `csr_matrix`
    - 長所：行ベクトルを取り出すのが高速，行列-ベクトル積が高速，`csr_matrix`同士の演算が高速．
    - 短所：列ベクトルを取り出すのが遅い，他のスパースマトリクスに変換するのが遅い．
3. `csc_matrix`
    - 長所：列ベクトルを取り出すのが高速，行列-ベクトル積が高速，`csc_matrix`同士の演算が高速．
    - 短所：行ベクトルを取り出すのが遅い，他のスパースマトリクスに変換するのが遅い．

インデックスの保持の仕方を考えるとある程度納得できるのではないかと思います．
用途に応じて使い分けられると良いですね．

<a id='basic_operation'></a>

## 3.5 スパースマトリクスの基本操作

スパースマトリクスの`np.ndarray`への変換は`.toarray()`で行うことができます．スパースマトリクスを上手く扱えずに困った時の必殺技は，`.toarray()`で`np.ndarray`に変換し`np.ndarray`として扱うことです (メモリに乗ればですが)．以降，計算結果の比較を行う際に，`np.ndarray`の方がわかりやすいことが多いため，表示する直前に`toarray()`を多用します．

In [12]:
print(A_ndarray)
print(A_csr)
print(A_csr.toarray())
print("np.array(A_csr)はダメ")
print(np.array(A_csr)) # np.array()をかぶせても上手く行かない

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
  (0, 2)	1
  (1, 1)	4
  (1, 2)	-1
  (2, 0)	2
  (2, 2)	-3
  (3, 3)	5
[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
np.array(A_csr)はダメ
  (0, 2)	1
  (1, 1)	4
  (1, 2)	-1
  (2, 0)	2
  (2, 2)	-3
  (3, 3)	5


**与えられたオブジェクトがスパースマトリクスなのか否かの判定**は，`sparse.issparse`関数で行うことができます．

In [13]:
print(sparse.issparse(1))
print(sparse.issparse("aaaaaaaaaa"))
print(sparse.issparse(A_ndarray))
print(sparse.issparse(A_csr))
print(sparse.issparse(A_csc))
print(sparse.issparse(A_coo))

False
False
False
True
True
True


`np.ndarray`と同様の方法で基本情報を取得することができます．ただし，`size`では全体の要素数ではなく**非ゼロ要素数**が返ってきます．

In [14]:
print(A_coo.ndim, A_csr.ndim, A_csc.ndim)
print(A_coo.shape, A_csr.shape, A_csc.shape)
print(A_coo.dtype, A_csr.dtype, A_csc.dtype)
print(A_coo.size, A_csr.size, A_csc.size)

2 2 2
(4, 4) (4, 4) (4, 4)
int64 int64 int64
6 6 6


`csr_matrix`・ `csc_matrix`では，要素へのアクセスやスライスもサポートしています．`np.ndarray`と同様の方法で行うことができます．行ベクトル・列ベクトルへのアクセスでは，スパースなベクトルとしての結果が返ってきます．

In [15]:
print(A_ndarray)
print(A_csc[0, 0], A_csc[0, 1], A_csc[0, 2], A_csc[0, 3])
print('csr_matrixで行ベクトル・列ベクトルにアクセス')
print(A_csr[1])
print(A_csr[:, 3])
print('csc_matrixで行ベクトル・列ベクトルにアクセス')
print(A_csc[1])
print(A_csc[:, 3])

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
0 0 1 0
csr_matrixで行ベクトル・列ベクトルにアクセス
  (0, 1)	4
  (0, 2)	-1
  (3, 0)	5
csc_matrixで行ベクトル・列ベクトルにアクセス
  (0, 1)	4
  (0, 2)	-1
  (3, 0)	5


`coo_matrix`では要素へのアクセスが禁じられています．**エラーが返ってきます．**

In [16]:
print(A_coo[1, 2])

TypeError: 'coo_matrix' object is not subscriptable

**行列の定数倍**は`np.ndarray`と同様にできます．`csr_matrix`・`csc_matrix`では，行・列毎の定数倍も行えます．

In [23]:
print(A_ndarray)
print((A_coo*4).toarray())
print((A_csr*4).toarray())
print((A_csc*4).toarray())
_A_csr = A_csr.copy() # コピー
_A_csr[0] *= 4
_A_csr[:, 2] *= -2
print(_A_csr.toarray())

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
[[  0   0   4   0]
 [  0  16  -4   0]
 [  8   0 -12   0]
 [  0   0   0  20]]
[[  0   0   4   0]
 [  0  16  -4   0]
 [  8   0 -12   0]
 [  0   0   0  20]]
[[  0   0   4   0]
 [  0  16  -4   0]
 [  8   0 -12   0]
 [  0   0   0  20]]
[[ 0  0 -8  0]
 [ 0  4  2  0]
 [ 2  0  6  0]
 [ 0  0  0  5]]


**定数和**はスパース性を破壊してしまうため**実行できません．**

In [24]:
print(A_csr+4)

NotImplementedError: adding a nonzero scalar to a sparse matrix is not supported

**スパースマトリクスとスパースマトリクスの和**は`np.ndarray`と同様にできます．異なるスパースマトリクス同士でもできます (が，同じタイプのスパースマトリクス同士で行うほうが高速です)．

In [29]:
print((A_csr+A_csr).toarray())
print((A_csc+A_csc).toarray())
print((A_coo+A_coo).toarray())
print((A_csr+A_csc).toarray())
print((A_coo+A_csc).toarray())

[[ 0  0  2  0]
 [ 0  8 -2  0]
 [ 4  0 -6  0]
 [ 0  0  0 10]]
[[ 0  0  2  0]
 [ 0  8 -2  0]
 [ 4  0 -6  0]
 [ 0  0  0 10]]
[[ 0  0  2  0]
 [ 0  8 -2  0]
 [ 4  0 -6  0]
 [ 0  0  0 10]]
[[ 0  0  2  0]
 [ 0  8 -2  0]
 [ 4  0 -6  0]
 [ 0  0  0 10]]
[[ 0  0  2  0]
 [ 0  8 -2  0]
 [ 4  0 -6  0]
 [ 0  0  0 10]]


`np.ndarray`と同様に**転置は`.T`**で行うことができます．

In [54]:
print(A_ndarray)
print(A_csc.toarray())
print(A_csc.T.toarray())

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
[[ 0  0  2  0]
 [ 0  4  0  0]
 [ 1 -1 -3  0]
 [ 0  0  0  5]]


**和**は`.sum()`で計算ができます．`np.ndarray`と同様に，`axis`キーワードを用いることができます．

In [23]:
print(np.sum(A_ndarray), A_coo.sum(), A_csr.sum(),  A_csc.sum())
print("各行ベクトルの和")
print(np.sum(A_ndarray, axis=0))
print(A_coo.sum(axis=0))
print(A_csr.sum(axis=0))
print(A_csc.sum(axis=0))
print("各列ベクトルの和")
print(np.sum(A_ndarray, axis=1))
print(A_coo.sum(axis=1))
print(A_csr.sum(axis=1))
print(A_csc.sum(axis=1))

8 8 8 8
各行ベクトルの和
[ 2  4 -3  5]
[[ 2  4 -3  5]]
[[ 2  4 -3  5]]
[[ 2  4 -3  5]]
各列ベクトルの和
[ 1  3 -1  5]
[[ 1]
 [ 3]
 [-1]
 [ 5]]
[[ 1]
 [ 3]
 [-1]
 [ 5]]
[[ 1]
 [ 3]
 [-1]
 [ 5]]


上の結果を見ると，`axis`キーワードを指定した場合，`np.ndarray`の挙動とスパースマトリクスの挙動がやや異なっています．実は，スパースマトリクスの`sum`は，`axis`をキーワードを用いた場合`np.matrix`を返します．

In [87]:
print(type(np.sum(A_ndarray, axis=0)))
print(type(A_csr.sum(axis=0)))

print(type(np.sum(A_ndarray, axis=1)))
print(type(A_csr.sum(axis=1)))

<class 'numpy.ndarray'>
<class 'numpy.matrix'>
<class 'numpy.ndarray'>
<class 'numpy.matrix'>


`np.matrix`は行列用のクラスで常に次元は2次元になります．それが先ほどの挙動の違いの理由です．

残念なことに，`np.matrix`と`np.ndarray`の間で計算を行うと，`np.ndarray`同士での演算結果とは異なることがほとんどです．以下のセルがその例を示しています．下のセルでは，`np.ndarray`と`np.matrix`の間で演算が発生した時に，`np.matrix`準拠の結果になっています．計算の結果返ってくる型は`np.matrix`です．


In [37]:
print("ndarrayとndarrayで+")
print(np.sum(A_ndarray, axis=1)+np.sum(A_ndarray, axis=1))
print("ndarrayとmatrixで+")
print(np.sum(A_ndarray, axis=1)+A_csr.sum(axis=1))
print(type(np.sum(A_ndarray, axis=1)+A_csr.sum(axis=1)))

print("ndarrayとndarrayで*")
print(np.sum(A_ndarray, axis=1)*np.sum(A_ndarray, axis=1))
print("ndarrayとmatrixで*")
print(np.sum(A_ndarray, axis=1)*A_csr.sum(axis=1))
print(type(np.sum(A_ndarray, axis=1)*A_csr.sum(axis=1)))
print("matrixとndarrayで*") # 上と順序を逆にする
print(A_csr.sum(axis=1)*np.sum(A_ndarray, axis=1))
print(type(A_csr.sum(axis=1)*np.sum(A_ndarray, axis=1)))

ndarrayとndarrayで+
[ 2  6 -2 10]
ndarrayとmatrixで+
[[ 2  4  0  6]
 [ 4  6  2  8]
 [ 0  2 -2  4]
 [ 6  8  4 10]]
<class 'numpy.matrix'>
ndarrayとndarrayで*
[ 1  9  1 25]
ndarrayとmatrixで*
[[36]]
<class 'numpy.matrix'>
matrixとndarrayで*
[[ 1  3 -1  5]
 [ 3  9 -3 15]
 [-1 -3  1 -5]
 [ 5 15 -5 25]]
<class 'numpy.matrix'>


`np.matrix`よりも`np.ndarray`の方が使いやすいということも多いです．`np.matrix`に`np.ndarray`を被せることで`np.ndarray`に変換できます．

In [88]:
print(A_csr.sum(axis=1))
print(type(A_csr.sum(axis=1)))
print(np.array(A_csr.sum(axis=1)))
print(type(np.array(A_csr.sum(axis=1))))

[[ 1]
 [ 3]
 [-1]
 [ 5]]
<class 'numpy.matrix'>
[[ 1]
 [ 3]
 [-1]
 [ 5]]
<class 'numpy.ndarray'>


スパースマトリクスでは`.toarray()`がありましたが，`np.matrix`にはありません．**エラーが出ます．**

In [89]:
print(A_csr.sum(axis=1).toarray())

AttributeError: 'matrix' object has no attribute 'toarray'

`np.matrix`における`*`は，**要素ごとの積ではなく行列積**です．これはスパースマトリクスでも同じです．**スパースマトリクスで要素毎の積**を行いたい場合は，`.multiply`を使います．

In [14]:
print("スパースマトリクスの*は行列積(dot)")
print(np.dot(A_ndarray, A_ndarray.T))
print((A_csc*A_csc.T).toarray())

print("スパースマトリクスの要素積はmultiply")
print(A_ndarray*A_ndarray)
print(A_csc.multiply(A_csc).toarray())

スパースマトリクスの*は行列積(dot)
[[ 1 -1 -3  0]
 [-1 17  3  0]
 [-3  3 13  0]
 [ 0  0  0 25]]
[[ 1 -1 -3  0]
 [-1 17  3  0]
 [-3  3 13  0]
 [ 0  0  0 25]]
スパースマトリクスの要素積はmultiply
[[ 0  0  1  0]
 [ 0 16  1  0]
 [ 4  0  9  0]
 [ 0  0  0 25]]
[[ 0  0  1  0]
 [ 0 16  1  0]
 [ 4  0  9  0]
 [ 0  0  0 25]]


スパースマトリクスと`np.ndarray`のベクトルで積を計算したい場合があります (`np.dot(A_ndarray, b)`に対応)．そのまま`np.dot`を使ってみると，次のような結果になります．一体どのような計算がされると次のような結果になるのでしょうか？

In [65]:
b = np.arange(4)
print(A_ndarray)
print(np.dot(A_ndarray, b))
print(np.dot(A_csr, b))
print(type(np.dot(A_csr, b)))

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
[ 2  2 -6 15]
[<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>
 <4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>
 <4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>
 <4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>]
<class 'numpy.ndarray'>


答え合わせです．`np.dot(A_csr, b)`の演算結果は，要素数4の`np.ndarray`で，各要素が`csr_matrix`となっています．中身を覗いてみましょう．各要素は，`A_csr`に`b[i]`を掛けた(定数倍)結果になっています．予想外です．

In [67]:
print(A_ndarray)
for i in range(4):
    print(np.dot(A_csr, b)[i].toarray())

[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[ 0  0  1  0]
 [ 0  4 -1  0]
 [ 2  0 -3  0]
 [ 0  0  0  5]]
[[ 0  0  2  0]
 [ 0  8 -2  0]
 [ 4  0 -6  0]
 [ 0  0  0 10]]
[[ 0  0  3  0]
 [ 0 12 -3  0]
 [ 6  0 -9  0]
 [ 0  0  0 15]]


スパースマトリクスと`np.ndarray`のベクトルの積 (`np.dot(A_ndarray, b)`に対応)は，`*`で計算できます．ややこしいですね．

In [97]:
print(np.dot(A_ndarray, b))
print((A_csr*b))

[ 2  2 -6 15]
[ 2  2 -6 15]


`np.ndarray`同様に，ここで紹介した関数以外にも多くの関数があります．より詳しく知りたい方は公式のリファレンス (https://docs.scipy.org/doc/scipy/reference/sparse.html )をご覧ください．

<a id='summary'></a>
## 3.6 まとめ

- スパースマトリクスは0が多い行列を効率良く扱うためのクラス(データ構造)です．
- 非ゼロ要素の値と非ゼロ要素のインデックスを保持してデータを表現します．
- インデックスの保持の仕方には色々あり，それによって`coo_matrix`や`csr_matrix`，`csc_matrix`といったようにいくつかのヴァリエーションが存在します．インデックスの保持の仕方によって，長所短所があります．
- `np.ndarray`で行うことができた様々な演算の多くが同様に実行できます．ただし，`*`のように，いくつかの演算は`np.ndarray`とは異なる演算結果となるため注意が必要です．また，`np.matrix`を返すこともあり，それにも注意が必要です．
- 操作に困ってどうしようもないときの最終奥義は，`.toarray()`で`np.ndarray`に変換し，`np.ndarray`として扱ってしまうことです．